In [5]:
import pandas as np
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

In [6]:
with open('/time_series_1.pickle', 'rb') as handle:
  time_series_1 = pickle.load(handle)

with open('/time_series_2.pickle', 'rb') as handle:
  time_series_2 = pickle.load(handle)

with open('/y.pickle', 'rb') as handle:
  y = pickle. load(handle)

In [7]:
time_series_1

array([[30, 29, 29, ..., 36, 35, 50],
       [31, 31, 30, ..., 37, 32, 31],
       [28, 28, 28, ..., 40, 29, 38],
       ...,
       [28, 28, 28, ..., 38, 28, 36],
       [28, 28, 29, ..., 31, 36, 51],
       [31, 31, 30, ..., 35, 38, 24]], dtype=uint8)

In [8]:
time_series_2

array([[84, 72, 93, ...,  2,  2,  1],
       [71, 83, 81, ...,  1,  1,  1],
       [78, 71, 82, ...,  0,  0,  0],
       ...,
       [83, 72, 95, ...,  0,  0,  0],
       [84, 68, 92, ...,  1,  0,  0],
       [81, 71, 98, ...,  0,  0,  0]], dtype=uint8)

In [9]:
y

array([1., 2., 3., ..., 3., 3., 1.])

In [10]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split

# Combine time series data
X = np.stack((time_series_1, time_series_2), axis=-1)

# Convert y to one-hot encoding
y = tf.keras.utils.to_categorical(y, num_classes=4)

In [11]:
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify = y)

In [12]:
# Define batch size and shuffle buffer size
batch_size = 32
shuffle_buffer_size = len(y_train)

In [13]:
# Create a TensorFlow dataset from the train set
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = train_dataset.shuffle(shuffle_buffer_size).batch(batch_size)

# Create a TensorFlow dataset from the test set
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))
test_dataset = test_dataset.batch(batch_size)

In [14]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(filters=32, kernel_size=5, activation='relu', input_shape=(5000, 2)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.SimpleRNN(units=64, dropout=0.2, recurrent_dropout=0.2, return_sequences=True),
    tf.keras.layers.LSTM(units=128, dropout=0.2, recurrent_dropout=0.2, return_sequences=True),
    tf.keras.layers.GRU(units=64, dropout=0.2, recurrent_dropout=0.2),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(4, activation='softmax')
])

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# For creating the checkpoint for out model so that we can save the optimal model.
filepath="/content/drive/MyDrive/workspace/sarthak/complex_model.hdf5"

checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
early_stop = EarlyStopping(monitor="val_loss", patience=5, mode="min", verbose=1, restore_best_weights=True)

callbacks_list = [checkpoint, early_stop]

In [15]:
history = model.fit(train_dataset, epochs=10, validation_data=test_dataset, verbose = 1, callbacks=callbacks_list)

Epoch 1/10
100/100 [==============================] - ETA: 0s - loss: 1.3863 - accuracy: 0.2681 
Epoch 1: val_loss improved from inf to 1.38808, saving model to /content/drive/MyDrive/workspace/sarthak/complex_model.hdf5
100/100 [==============================] - 1184s 12s/step - loss: 1.3863 - accuracy: 0.2681 - val_loss: 1.3881 - val_accuracy: 0.2500
Epoch 2/10
100/100 [==============================] - ETA: 0s - loss: 1.3631 - accuracy: 0.3063 
Epoch 2: val_loss did not improve from 1.38808
100/100 [==============================] - 1167s 12s/step - loss: 1.3631 - accuracy: 0.3063 - val_loss: 1.3992 - val_accuracy: 0.2600
Epoch 3/10
100/100 [==============================] - ETA: 0s - loss: 1.3098 - accuracy: 0.3606 
Epoch 3: val_loss improved from 1.38808 to 1.32165, saving model to /content/drive/MyDrive/workspace/sarthak/complex_model.hdf5
100/100 [==============================] - 1167s 12s/step - loss: 1.3098 - accuracy: 0.3606 - val_loss: 1.3216 - val_accuracy: 0.3300
Epoch 4/

In [23]:
 test_loss, test_acc = model.evaluate(test_dataset)
print('Test accuracy',test_acc)
print('Test loss',test_loss)

25/25 [==============================] - 32s 1s/step - loss: 0.7109 - accuracy: 0.7200
Test accuracy 0.7200000286102295
Test loss 0.7108709216117859
